# Set up

In [2]:
import torchreid
from torchreid.data.datasets.image.til2023_cv_test import Til2023CvTest
torchreid.data.register_image_dataset('til2023_cv_test', Til2023CvTest)

/notebooks/deep-person-reid/torchreid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [3]:
datamanager = torchreid.data.ImageDataManager(
    root='reid-data',
    sources='til2023_cv_test',
    transforms = ""
)

Building train transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x128
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
TRAIN:::  [('/notebooks/deep-person-reid/reid-data/til2023_cv_dataset/Train1/00066_5618.png', 66, 0), ('/notebooks/deep-person-reid/reid-data/til2023_cv_dataset/Train1/00175_3145.png', 175, 0), ('/notebooks/deep-person-reid/reid-data/til2023_cv_dataset/Train1/00158_5614.png', 158, 0), ('/notebooks/deep-person-reid/reid-data/til2023_cv_dataset/Train1/00060_4026.png', 60, 0), ('/notebooks/deep-person-reid/reid-data/til2023_cv_dataset/Train1/00156_4167.png', 156, 0), ('/notebooks/deep-person-reid/reid-data/til2023_cv_dataset/Train1/00107_3907.png', 107, 0), ('/notebooks/deep-person-reid/reid-data/til2023_cv_dataset/Train1/00067_2344.png', 67, 0), ('/

# Assemble Model

## model 1 - resnet50

In [ ]:
model = torchreid.models.build_model(
    name="resnet50",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

## mode 2 - resnet50mid

## model 4 - osnet_ain_x0_1

In [ ]:
model = torchreid.models.build_model(
    name="osnet_ain_x1_0",
    num_classes=datamanager.num_train_pids,
    loss="triplet",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0015,
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

## model 5 - osnet_ain_x0_25

In [ ]:
model = torchreid.models.build_model(
    name="osnet_ain_x0_25",
    num_classes=datamanager.num_train_pids,
    loss="softmax",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

## model - osnet ibn x1_0

In [4]:
model = torchreid.models.build_model(
    name="osnet_ibn_x1_0",
    num_classes=datamanager.num_train_pids,
    loss="triplet",
    pretrained=True
)

model = model.cuda()

optimizer = torchreid.optim.build_optimizer(
    model,
    optim="adam",
    lr=0.0003
)

scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler="single_step",
    stepsize=20
)

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_ibn_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


# Assemble engine, load weights

In [5]:
engine = torchreid.engine.ImageTripletEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True
)

In [6]:
from torchreid.utils import load_pretrained_weights

In [7]:
weight_path = '/notebooks/deep-person-reid/log/til_osnet_ibn_x1_0_with_pretrained/model/model.pth.tar-10-95.7-460'

In [8]:
load_pretrained_weights(model, weight_path)

Successfully loaded pretrained weights from "/notebooks/deep-person-reid/log/til_osnet_ibn_x1_0_with_pretrained/model/model.pth.tar-10-95.7-460"


In [ ]:
_,distmat, queries_and_galley = engine.run(
    save_dir="log/til_resnet50",
    max_epoch=4,
    eval_freq=1,
    print_freq=50,
    test_only=True
)

##### Evaluating til2023_cv_test (source) #####
Extracting features from query set ...
HELLO HERE ARE UR PIDS AND CAMIDS: 
[0 0 0 ... 0 0 0] [0 0 0 ... 0 0 0]
Done, obtained 1600-by-512 matrix
Extracting features from gallery set ...
IOError incurred when reading "/notebooks/deep-person-reid/reid-data/til2023_cv_test/Test1/.ipynb_checkpoints". Will redo. Don't worry. Just chill.
IOError incurred when reading "/notebooks/deep-person-reid/reid-data/til2023_cv_test/Test1/.ipynb_checkpoints". Will redo. Don't worry. Just chill.
IOError incurred when reading "/notebooks/deep-person-reid/reid-data/til2023_cv_test/Test1/.ipynb_checkpoints". Will redo. Don't worry. Just chill.
IOError incurred when reading "/notebooks/deep-person-reid/reid-data/til2023_cv_test/Test1/.ipynb_checkpoints". Will redo. Don't worry. Just chill.
IOError incurred when reading "/notebooks/deep-person-reid/reid-data/til2023_cv_test/Test1/.ipynb_checkpoints". Will redo. Don't worry. Just chill.
IOError incurred when read

# post process results

In [ ]:
from PIL import Image

In [ ]:
def visualise_matches(d, q, index):
    query = q[0][index][0]
    distmat = d[index]
    gallery = q[1]
    
    temp = zip(distmat, gallery)
    print("#SUS ", query)
    im=Image.open(query)
    im.show()
    for i in sorted(temp):
        if query[-8:-4] in i[1][0][-13:-8]:
            print("#CANDIDATES: ", i)
            im=Image.open(i[1][0])
            im.show()

In [ ]:
for i in range(755,788):
    visualise_matches(distmat, queries_and_galley, i)

In [ ]:
def visualise_sorts(d, q, index):
    query = q[0][index][0]
    distmat = d[index]
    gallery = q[1]
    
    temp = zip(distmat, gallery)
    print("#SUS ", query)
    im=Image.open(query)
    im.show()
    count = 0
    for i in sorted(temp):
        if count <= 50:
            print("#SORTS: ", i)
            im=Image.open(i[1][0])
            im.show()
        
        if query[-8:-4] in i[1][0][-13:-8]:
            print("#CANDIDATES: ", i)
            im=Image.open(i[1][0])
            im.show()
        
        count += 1

In [ ]:
visualise_sorts(distmat, queries_and_galley, 10)

In [ ]:
def output_matches(d, q, index):
    query = q[0][index][0]
    distmat = d[index]
    gallery = q[1]
    
    temp = zip(distmat, gallery)
    # print(query)
    # im=Image.open(query)
    # im.show()
    
    output = {}
    for i in sorted(temp):
        if query[-8:-4] in i[1][0][-13:-8]:
            if i[0] < 460 and output == {}:
                output[int(i[1][0][-8:-4])] = 1
            else:
                output[int(i[1][0][-8:-4])] = 0
                
    return output

In [ ]:
import pandas as pd
from pandas import DataFrame
path = "/notebooks/deep-person-reid/reid-data/results (5).csv"
df = pd.read_csv(path)
df.head(10)

In [ ]:
for i in range(1600):
    output = output_matches(distmat, queries_and_galley, i)
    for j in output.keys():
        df.at[j,"class"] = output[int(j)]

In [ ]:
df.head(40)

In [ ]:
df.to_csv("/notebooks/file_transfer/submission_20.csv")